<a href="https://colab.research.google.com/github/anysabele/Projeto-Bootcamp-ML-Avanti-2026.1/blob/main/Equipe_2ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação de Resíduos Recicláveis

______________
[DATASET AQUI](https://www.kaggle.com/datasets/alistairking/recyclable-and-household-waste-classification)
_______

ALUNOS:
1. Alan Correia Timbó
2. Ana Marly do Couto da Silva
3. Anysabele de Paula Barbosa Santos
4. Elian Ramon de Sarges Rodrigues
5. Jaqueline de Lima Campos
6. Josyele Moura de Alencar
7. José Yago Lisboa da Silva
8. João Inácio de Souza Neto
9. Juliana Leite Pinto Magalhães
________
[MODELO PORFESSOR ETAPA 01](https://colab.research.google.com/drive/1X3gLPXrGM3uMzpYMHIcmpvBShHATLQL9?usp=sharing)

## Carregamento do dataset

In [2]:
# Imports

!pip install opendatasets
!pip install imagehash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 5.0 MB/s eta 0:00:00


In [3]:
import opendatasets as od
import pandas as pd

od.download("https://www.kaggle.com/datasets/alistairking/recyclable-and-household-waste-classification")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: Anyuau
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/alistairking/recyclable-and-household-waste-classification


100%|██████████| 920M/920M [00:08<00:00, 117MB/s]


In [4]:
!ls

recyclable-and-household-waste-classification  sample_data


## Criando dataframe

In [5]:
import cv2
import os
import imagehash
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image

In [6]:
root_dir = './recyclable-and-household-waste-classification/images/images'

In [10]:
count_corrupted = 0
corrupted = list()

dataframe_list = list()

for dirpath, dirnames, filenames in os.walk(root_dir):
    # Extract the category folder. Assumes the structure is root_dir/category/sub_category/image.jpg
    # The 'folder' in the original code was the immediate child of root_dir (e.g., 'plastic_shopping_bags')
    relative_path = os.path.relpath(dirpath, root_dir)
    path_parts = relative_path.split(os.sep)

    category_folder = None
    if len(path_parts) > 0 and path_parts[0] != '.': # path_parts[0] could be '.' if dirpath is root_dir itself
        category_folder = path_parts[0]

    # Only process files if a category folder could be determined (i.e., not the root_dir itself)
    if category_folder is None:
        continue

    for image_name in filenames:
        img_dir = os.path.join(dirpath, image_name)

        # Check if the file has an extension
        if '.' in image_name:
            name_parts = image_name.rsplit('.', 1)
            if len(name_parts) == 2:
                image_format = name_parts[1].lower()
            else:
                continue # Skip files without a proper extension
        else:
            continue # Skip files without an extension

        img = cv2.imread(img_dir)

        if img is not None:
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(img_rgb)
            img_hash = imagehash.phash(img_pil)
            width, height, channels = img.shape
            img_corrupted = False
        else:
            count_corrupted += 1
            corrupted.append(img_dir)
            img_corrupted = True
            img_hash, width, height, channels = None, None, None, None

        dataframe_list.append([img_dir, img_corrupted, img_hash, image_format, width, height, channels, category_folder])

In [11]:
dataframe_list

[['./recyclable-and-household-waste-classification/images/images/glass_beverage_bottles/default/Image_139.png',
  False,
  array([[ True,  True,  True, False,  True, False, False,  True],
         [False, False,  True,  True, False, False, False, False],
         [ True, False, False,  True, False,  True,  True, False],
         [ True, False, False,  True, False, False,  True,  True],
         [ True, False, False,  True, False,  True,  True, False],
         [False,  True,  True, False,  True,  True,  True, False],
         [ True, False, False,  True, False,  True,  True, False],
         [False,  True,  True, False,  True, False, False,  True]]),
  'png',
  256,
  256,
  3,
  'glass_beverage_bottles'],
 ['./recyclable-and-household-waste-classification/images/images/glass_beverage_bottles/default/Image_12.png',
  False,
  array([[ True, False,  True,  True, False, False, False, False],
         [ True,  True, False,  True, False, False,  True,  True],
         [ True,  True, False,

In [12]:
print(f'Total de Imagens Corrompidas: {count_corrupted}\nImagens corrompidas {corrupted}')

Total de Imagens Corrompidas: 0
Imagens corrompidas []


In [13]:
df = pd.DataFrame(columns=['image_path', 'corrupted', 'image_hash', 'image_format', 'width', 'height', 'channels', 'label'], data = dataframe_list)

In [14]:
df.head()

,image_path,corrupted,image_hash,image_format,width,height,channels,label
0,./recyclable-and-household-waste-classificatio...,False,e9309693966e9669,png,256,256,3,glass_beverage_bottles
1,./recyclable-and-household-waste-classificatio...,False,b0d3c696e64c9b2c,png,256,256,3,glass_beverage_bottles
2,./recyclable-and-household-waste-classificatio...,False,e99cb6699d368099,png,256,256,3,glass_beverage_bottles
3,./recyclable-and-household-waste-classificatio...,False,eb94946ad03d6be0,png,256,256,3,glass_beverage_bottles
4,./recyclable-and-household-waste-classificatio...,False,eb99e54694661a5a,png,256,256,3,glass_beverage_bottles


In [22]:
valid_ext = (".jpg", ".jpeg", ".png")

non_images = []

for root, dirs, files in os.walk(root_dir):
    for f in files:
        if not f.lower().endswith(valid_ext):
            non_images.append(os.path.join(root, f))

print("Arquivos que não são imagens:")
print(non_images)

Arquivos que não são imagens:
[]


In [24]:
from collections import Counter

formats = []

for root, dirs, files in os.walk(root_dir):
    for file in files:
        path = os.path.join(root, file)

        try:
            with Image.open(path) as img:
                formats.append(img.format)
        except:
            pass  # ignora arquivos que não são imagens

# Contagem dos formatos
format_counts = Counter(formats)

print("Formatos encontrados no dataset:")
print(format_counts)

Formatos encontrados no dataset:
Counter({'PNG': 15000})


In [20]:
print(f'Total de Imagens: {len(df)}')
print(f'Shape do DataFrame: {df.shape[0]} linhas, {df.shape[1]} colunas')

Total de Imagens: 15000
Shape do DataFrame: 15000 linhas, 8 colunas


* O dataset "Recyclable and Household Waste Classification" possui um total de 1500 imagens (todas em PNG) e 8 colunas, e nenhuma imagem corrompida.

In [25]:
df.describe().loc[['mean', 'std', 'min', 'max']]

,width,height,channels
mean,256.0,256.0,3.0
std,0.0,0.0,0.0
min,256.0,256.0,3.0
max,256.0,256.0,3.0


In [30]:
# Contar as ocorrências de cada classe na coluna 'label' do DataFrame df
df_classes = df['label'].value_counts().reset_index()

# Renomear as colunas para 'classe' e 'quantidade'
df_classes.columns = ["classe", "quantidade"]

df_classes

,classe,quantidade
0,glass_beverage_bottles,500
1,aerosol_cans,500
2,plastic_straws,500
3,aluminum_soda_cans,500
4,magazines,500
5,shoes,500
6,office_paper,500
7,cardboard_packaging,500
8,aluminum_food_cans,500
9,clothing,500
